In [ ]:
import pandas as pd
import re

#Funciones
def clean_text(text):
    text = text.upper()  # Convertir a mayúsculas
    text = re.sub(r"[^A-Z0-9, ]", "", text)  # Eliminar caracteres especiales excepto comas y espacios
    text = re.sub(r"\s+", " ", text).strip()  # Eliminar espacios extra
    return text

#NODOS
nodos_df = pd.read_excel("Base-inicial.xlsx",sheet_name='base')
nodos_df = nodos_df.dropna(subset=["Principales inversionistas","Organization Name"])
nodos_data = nodos_df[["id", "Organization Name", "Principales inversionistas"]].copy()
nodos_data["Organization Name"] = nodos_data["Organization Name"].apply(clean_text)
nodos_data["Principales inversionistas"] = nodos_data["Principales inversionistas"].apply(clean_text)
nodos_data.rename(columns={"Organization Name": "Label"}, inplace=True)
# Guardar el resultado limpio en un nuevo CSV
nodos_data.to_csv("nodos.csv", index=False)

#ARCOS
# Crear el DataFrame de arcos
arcos = []
for _, row in nodos_data.iterrows():
    target = row["Label"]
    inversionistas = row["Principales inversionistas"].split(", ")  # Separar inversionistas
    
    for investor in inversionistas:
        investor = investor.strip()
        if investor in nodos_data["Label"].values:  # Solo agrega si el inversor también está en la base
            arcos.append([investor, target, "directed"])  # Directed porque es una relación de inversión

# Convertir a DataFrame
df_arcos = pd.DataFrame(arcos, columns=["Source", "Target", "Type"])

# Agregar un ID único para cada arco
df_arcos.insert(0, "Id", range(1, len(df_arcos) + 1))

# Guardar en CSV
df_arcos.to_csv("arcos.csv", index=False)